In [1]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)

f = open('C:/Users/jedgo/Projects/Jupyter1/tf/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open ('C:/Users/jedgo/Projects/Jupyter1/tf/labels.txt')
raw_labels = f.readlines()
f.close()

vocab = set()
tokens = list(map(lambda x:(x.split(" ")), raw_reviews))
for sent in tokens:
    for word in sent:
        if(len(word) > 0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [6]:
import numpy
np.random.seed(1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.01
weights_1_2 = 0.2*np.random.random((hidden_size, 1)) - 0.01

correct,total = (0, 0)
for iter in range(iterations):

    for i in range(len(input_dataset) - 1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))

        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter: ' + str(iter) + ' Progress:' + progress[2:4] + '.' + progress[4:6] + '% Training Accuracy: ' + str(correct/float(total)) + '%')
    print()
correct, total = (0,0)
for i in range(len(input_dataset) - 1000, len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))

    if(np.abs(layer_2 - y) < 0.5 ):
        correct += 1
    total += 1
print("Test Accuracy: " + str(correct / float(total)))

Iter: 0 Progress:95.99% Training Accuracy: 0.446875%
Iter: 1 Progress:95.99% Training Accuracy: 0.6239583333333333%
Test Accuracy: 0.809


In [7]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('C:/Users/jedgo/Projects/Jupyter1/tf/reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)

random.shuffle(input_dataset)

alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.05) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size) * 0.2

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
    for target_i in range(len(review)):

        target_samples = [review[target_i]] + list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

        left_context = review[max(0,target_i-window):target_i]
        right_context = review[target_i+1:min(len(review),target_i+window)]

        layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context + right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1) * alpha

    if(rev_i % 250 == 0):
        sys.stdout.write('\rProgress: '+str(rev_i/float(len(input_dataset) * iterations)) + "    " + str(similar('terrible')))
    sys.stdout.write('\rProgress: '+str(rev_i/float(len(input_dataset) * iterations)))
print(similar('terrible'))
print(similar('beautiful'))

Progress: 0.99998[('terrible', -0.0), ('horrible', -2.8333245130553166), ('brilliant', -3.4094452819963372), ('pathetic', -3.7842204335098177), ('marvelous', -3.9523828162647683), ('superb', -4.049999769721949), ('phenomenal', -4.093017406376725), ('masterful', -4.1036601182325825), ('mediocre', -4.127554971541032), ('bad', -4.160464232938011)]
[('beautiful', -0.0), ('creepy', -3.2201093174104), ('lovely', -3.2217642292796893), ('spooky', -3.5581897144575665), ('glamorous', -3.5638721871949137), ('nightmarish', -3.6203188839548677), ('ravishing', -3.6361892551728277), ('drab', -3.6601312091355536), ('colorful', -3.6861603373283574), ('delightful', -3.6878145613272424)]


In [8]:
import numpy as np
norms = np.sum(weights_0_1 * weights_0_1,axis=1)
norms.resize(norms.shape[0],1)
normed_weights = weights_0_1 * norms

def make_sent_vect(words):
    indices = list(map(lambda x:word2index[x],filter(lambda x:x in word2index,words)))
    return np.mean(normed_weights[indices],axis = 0)

reviews2vectors = list()
for review in tokens:
    reviews2vectors.append(make_sent_vect(review))
reviews2vectors = np.array(reviews2vectors)

def most_similar_reviews(review):
    v =  make_sent_vect(review)
    scores = Counter()
    for i,val in enumerate(reviews2vectors.dot(v)):
        scores[i] = val
    most_similar = list()

    for idx,score in scores.most_common(3):
        most_similar.append(raw_reviews[idx][0:40])
    return most_similar
most_similar_reviews(['boring','awful'])

['i  m surprised how many people give this',
 'probably new zealands worst movie ever m',
 '  br    br   a friend of mine enjoys wat']

In [9]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([0.1, 0.2, 0.3])
c = np.array([-1, -0.5, 0])
d = np.array([0, 0, 0])

identity = np.eye(3)
print(identity)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [10]:
print(a.dot(identity))
print(b.dot(identity))
print(c.dot(identity))
print(d.dot(identity))

[1. 2. 3.]
[0.1 0.2 0.3]
[-1.  -0.5  0. ]
[0. 0. 0.]


In [11]:
this = np.array([2, 4, 6])
movie = np.array([10, 10, 10])
rocks = np.array([1, 1, 1])

print(this + movie + rocks)
print((this.dot(identity) + movie.dot(identity) + rocks))

[13 15 17]
[13. 15. 17.]


In [12]:
import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

word_vects = {}
word_vects['yankees'] = np.array([[0.,0.,0.]])
word_vects['bears'] = np.array([[0.,0.,0.]])
word_vects['braves'] = np.array([[0.,0.,0.]])
word_vects['red'] = np.array([[0.,0.,0.]])
word_vects['sox'] = np.array([[0.,0.,0.]])
word_vects['lose'] = np.array([[0.,0.,0.]])
word_vects['defeat'] = np.array([[0.,0.,0.]])
word_vects['beat'] = np.array([[0.,0.,0.]])
word_vects['tie'] = np.array([[0.,0.,0.]])

sent2output = np.random.rand(3,len(word_vects))

identity = np.eye(3)

layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['sox']
layer_2 = layer_1.dot(identity) + word_vects['defeat']

pred = softmax(layer_2.dot(sent2output))
print(pred)

[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


In [ ]:
alpha = 0.01
y = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0])

pred_delta = pred - y
layer_2_delta = pred_delta.dot(sent2output.T)
defeat_delta = layer_2_delta
layer_1_delta = layer_2_delta.dot(identity.T)
sox_delta = layer_1_delta
layer_0_delta = layer_1_delta.dot(identity.T)

word_vects['red'] -= layer_0_delta * alpha
word_vects['sox'] -= sox_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha
identity -= np.outer(layer_0,layer_1_delta) * alpha
identity -= np.outer(layer_1,layer_2_delta) * alpha
sent2output -= np.outer(layer_2,pred_delta) * alpha

In [14]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('C:/Users/jedgo/Projects/Testing/Chain/en/qa1_single-supporting-fact_train.txt', 'r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [22]:
import sys,random,math
from collections import Counter
import numpy as np
np.random.seed(1)

f = open('C:/Users/jedgo/Projects/Testing/Chain/en/qa1_single-supporting-fact_train.txt', 'r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


embed_size = 10

embed = (np.random.rand(len(vocab), embed_size) - 0.5) * 0.1

recurrent = np.eye(embed_size)

start = np.zeros(embed_size)

decoder = (np.random.rand(embed_size,len(vocab)) - 0.5) * 0.1

one_hot = np.eye(len(vocab))

def predict(sent):

    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    preds = list()
    for target_i in range(len(sent)):

        layer = {}

        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))

        loss += -np.log(layer['pred'][sent[target_i]])

        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]

        layers.append(layer)
    return layers, loss

for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers, loss = predict(sent)

    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            if(layer_idx == len(layers) - 1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
for layer_idx,layer in enumerate(layers[1:]):

    decoder -= np.outer(layers[layer_idx]['hidden'],layer['output_delta']) * alpha / float(len(sent))

    embed_idx = sent[layer_idx]
    embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
    recurrent -= np.outer(layers[layer_idx]['hidden'],layer['hidden_delta']) * alpha / float(len(sent))

if(iter % 1000 == 0):
    print("Perplexity:" + str(np.exp(loss/len(sent))))